# Reducing Congestion and load balancing - Implementation Based on QUBO Formulation

Cities often face traffic congestion during major urban events such as concerts, sports games, festivals, or political gatherings. These events lead to sudden increases in traffic volume, which traditional traffic management systems struggle to handle. Unmanaged congestion can result in longer travel times, increased carbon emissions, public dissatisfaction, and challenges for emergency vehicles trying to navigate crowded streets.


Here, we explain the problem setup, formulation, and implementation of the present program for congestion reduction based on the QUBO formulation in the following order:

- 1\. [Problem setup](#1)
- 2\. [Formulation](#2)
  - 2.1\. [Concept](#2_1)
  - 2.2\. [Objective function](#2_2)
- 3\. [Implementation of rerouting to reduce congestion](#3)


<a id="1"></a>.
## 1\. Problem setup

The problem addressed in this program involves managing and reducing traffic congestion in an urban setting by optimizing the routes of multiple cars. Given a road network, each car has a set of alternative routes it can take. The goal is to minimize traffic congestion by finding an optimal route for each car such that road usage is balanced, reducing the number of shared roads between cars. The program leverages both classical and quantum computing techniques, particularly using a Quantum Unconstrained Binary Optimization (QUBO) formulation to model the problem of congestion reduction.

<a id="2"></a>

## 2\. Formulation

<a id="2_1"></a>

### 2.1\. Concept
Traffic congestion occurs when multiple cars share the same road, leading to overcrowding, delays, and inefficiencies. The aim is to route cars in such a way that the overlap of their routes on shared roads is minimized, and an even distribution of traffic is achieved across the available routes.

This is modeled as an optimization problem, where:

* Nodes represent intersections in the road network.
* Edges represent roads between intersections.
* Car routes are sets of edges (roads) connecting start and end points in the road network.
* Objective is to minimize the overlap (shared roads) between car routes.


### 2.2\. Objective Function
### 1. [Problem setup](#1)

The problem addressed in this program involves managing and reducing traffic congestion in an urban setting by optimizing the routes of multiple cars. Given a road network, each car has a set of alternative routes it can take. The goal is to minimize traffic congestion by finding an optimal route for each car such that road usage is balanced, reducing the number of shared roads between cars. The program leverages both classical and quantum computing techniques, particularly using a Quantum Unconstrained Binary Optimization (QUBO) formulation to model the problem of congestion reduction.

### 2. [Formulation](#2)

#### 2.1. [Concept](#2_1)

Traffic congestion occurs when multiple cars share the same road, leading to overcrowding, delays, and inefficiencies. The aim is to route cars in such a way that the overlap of their routes on shared roads is minimized, and an even distribution of traffic is achieved across the available routes.

This is modeled as an optimization problem, where:
- **Nodes** represent intersections in the road network.
- **Edges** represent roads between intersections.
- **Car routes** are sets of edges (roads) connecting start and end points in the road network.
- **Objective** is to minimize the overlap (shared roads) between car routes.

#### 2.2. [Objective function](#2_2)

The objective function aims to minimize the number of intersections (shared roads) among the cars. In the QUBO formulation:
- Binary variables $q_{ij}$ represent whether a car $i$ chooses route $j$.
- The cost function penalizes the use of shared roads between two cars by increasing the energy of the solution based on the number of intersections they share.

The QUBO objective function is designed to minimize:
- The total number of shared roads among different cars.
- The number of alternative routes a single car selects simultaneously (penalty for choosing multiple routes).

The QUBO matrix $Q$ is constructed with off-diagonal elements representing the penalty for two different cars using overlapping roads, and diagonal elements representing the penalty for selecting multiple routes for the same car.



<a id="3"></a>
## 3\. Implementation of re-routing to reduce congestion



### Preparation

First, we import the necessary Python libraries and set the API token to access the quantum computer

In [17]:
!pip install dwave-system

In [18]:
!export DWAVE_TOKEN=DEV-6f2f442b73ea9653b5c1f0a0af8b4864c35eb9c9

<a id="3.1"></a>

### 3.2. Writing Input File

This input file is structured to provide the necessary data for simulating traffic flow on a road network with nodes (intersections) and edges (roads between intersections). It includes information about the number of nodes, how the nodes are connected, and the routes taken by cars. Additionally, it provides alternative routes for each car to assist with the optimization problem.

#### 1. **Number of Nodes**
The first line specifies the total number of nodes (intersections) in the road network.

```plaintext
15 #how many nodes (corners where streets collide) (N)
```

- `15`: Represents the total number of nodes, i.e., intersections in the network.

#### 2. **Node Connections**
The next `N` lines (in this case, 15) describe the connections between the nodes. Each line corresponds to a node (identified by its index) and lists the nodes to which it is directly connected (neighboring intersections).

```plaintext
1 5 #next N lines have the nodes each node is connected to
0 2 6
1 3 7
2 4 8
3 9
0 6 10
1 5 7 11
2 6 8 12
3 7 9 13
4 8 14
5 11
6 10 12
7 11 13
8 12 14
9 13
```

- For example:
  - Node 0 is connected to nodes 1 and 5.
  - Node 1 is connected to nodes 0, 2, and 6.

Each node is identified by its index, starting from `0` to `14`, and is followed by the list of nodes to which it has a direct connection (road).

#### 3. **Number of Cars**
The next line specifies the total number of cars being considered in the simulation.

```plaintext
3 #how many cars (X)
```

- `3`: Indicates that there are three cars in the system.

#### 4. **Original Routes**
The next `X` lines (in this case, 3 lines) specify the original routes for each car. Each route is a sequence of node IDs representing the path a car takes through the network.

```plaintext
0-1-6-11-12-13-14 #next X lines have the original routes of cars (each number is the node ID)
5-10-11-12
2-7-12-13-14
```

- Car 0's route: Starts at node 0, moves through nodes 1, 6, 11, 12, 13, and ends at node 14.
- Car 1's route: Starts at node 5, moves through nodes 10, 11, and 12.
- Car 2's route: Starts at node 2, moves through nodes 7, 12, 13, and ends at node 14.

#### 5. **Alternate Routes**
The remaining lines describe alternative routes for the cars, allowing the algorithm to explore other possible paths to alleviate congestion. Each alternate route is associated with a car, specified by the car index and the sequence of node IDs for the alternate route.

```plaintext
#alternate routes
0 0-1-2-3-4-9-14
0 0-5-6-7-8-13-14
1 5-6-7-12
1 5-6-11-12
2 2-3-4-9-14
2 2-3-8-13-14
```

- For example:
  - Car 0 has two alternate routes:
    1. `0-1-2-3-4-9-14`
    2. `0-5-6-7-8-13-14`
  - Car 1 has two alternate routes:
    1. `5-6-7-12`
    2. `5-6-11-12`
  - Car 2 has two alternate routes:
    1. `2-3-4-9-14`
    2. `2-3-8-13-14`

Each alternate route provides possible detours to reduce traffic congestion. The car index is followed by the route (a sequence of nodes) the car can take instead of its original route.


In [19]:
%%writefile example.txt
15 #how many nodes (corners where streets collide) (N)
1 5 #next N lines have the nodes each node is connected to
0 2 6
1 3 7
2 4 8
3 9
0 6 10
1 5 7 11
2 6 8 12
3 7 9 13
4 8 14
5 11
6 10 12
7 11 13
8 12 14
9 13

3 #how many cars (X)
0-1-6-11-12-13-14 #next X lines have the original routes of cars (each number is the node ID)
5-10-11-12
2-7-12-13-14


#alternate routes
0 0-1-2-3-4-9-14
0 0-5-6-7-8-13-14
1 5-6-7-12
1 5-6-11-12
2 2-3-4-9-14
2 2-3-8-13-14

Overwriting example.txt


### Importing the necessary modules from dwave for quantum computing

In [20]:
from dwave.system import DWaveSampler
from dwave.system.composites import EmbeddingComposite

###Some Helper Functions

In [21]:
# Function to return unique elements in an array
def getUniqueArr(arr):
    ansArr = []  # Stores the unique elements
    for i in range(0, len(arr)):
        if(arr[i] not in ansArr):  # If the element is not in the ansArr, add it
            ansArr.append(arr[i])
    return ansArr  # Return the array with unique elements

# Function to find intersections (shared roads) between two car routes
def getIntersections(route1, route2):
    intersections = []  # Stores common roads between the two routes
    for i in range(0, len(route1)):
        if (route1[i] in route2):  # Check if a road in route1 is present in route2
            intersections.append(route1[i])
    return intersections  # Return the intersections found

### Function to create a QUBO matrix based on car routes and the number of alternate routes

In [22]:
def getQuboMatrix(carRoutes, numOfAlternateRoutes):
    Q = {}  # Dictionary to hold the QUBO matrix (symmetric, square matrix)

    # Initialize the QUBO matrix with zero values
    for i in range(len(carRoutes)*numOfAlternateRoutes):
        for j in range(len(carRoutes)*numOfAlternateRoutes):
            Q.update({(i, j): 0})

    carRouteIntersections = [0] * len(carRoutes)  # Stores the number of intersections for each car's routes
    for i in range(len(carRoutes)):
        for j in range(numOfAlternateRoutes):
            carIntersections = []  # Collects intersections for the current route
            for k in range(i + 1, len(carRoutes)):
                for m in range(numOfAlternateRoutes):
                    intersections = getIntersections(carRoutes[i][j], carRoutes[k][m])
                    # Add intersection costs for different cars
                    Q.update({(i*numOfAlternateRoutes + j, k*numOfAlternateRoutes + m): len(intersections) * 2})
                    carIntersections.extend(intersections)
            # Add the cost for choosing a route with multiple intersections
            Q.update({(i*numOfAlternateRoutes+j, i*numOfAlternateRoutes+j): len(carIntersections)})
            # Update the number of unique intersections for the car
            carRouteIntersections[i] += len(getUniqueArr(carIntersections))

    # Setting the penalty lambda to the maximum number of intersections
    lam = max(carRouteIntersections)

    # Iterate over all routes to update the diagonal and off-diagonal elements
    for i in range(0, len(carRoutes)):
        for j in range(0, numOfAlternateRoutes):
            # Penalize choosing multiple alternate routes for the same car
            for k in range(j+1, numOfAlternateRoutes):
                Q.update({(i*numOfAlternateRoutes + j, i*numOfAlternateRoutes + k): lam * 2})
            # Subtract lambda from the diagonal values to adjust penalties
            Q.update({(i*numOfAlternateRoutes + j, i*numOfAlternateRoutes + j): (Q[(i*numOfAlternateRoutes + j, i*numOfAlternateRoutes + j)] - lam)})

    return Q


### Class representing a Node in the traffic network (intersections)

In [23]:
class Node:
    def __init__(self, nodeID):
        self.__nodeID = nodeID
        self.__edges = []

    def getID(self):
        return self.__nodeID  # Return the node ID

    def getEdges(self):
        return self.__edges  # Return the list of connected edges

    # Check if the node has a specific edge
    def hasEdge(self, edge):
        for i in range(len(self.__edges)):
            if(self.__edges[i].getID() == edge.getID()):
                return True
        return False

    # Add an edge to the node if it doesn't already exist
    def addEdge(self, edge):
        if(self.hasEdge(edge)):
            return False
        self.__edges.append(edge)  # Append edge to this node
        edge.addNode(self)  # Add this node to the edge
        return True

    # Check if this node is equal to another node (by ID)
    def equals(self, node):
        if(node.getID() == self.getID()):
            return True
        return False

    # Get the edge connecting this node to another node
    def getEdgeWithNode(self, node):
        for i in range(len(self.__edges)):
            if(self.__edges[i].getOtherNode(self).equals(node)):
                return self.__edges[i]
        return None


### Class representing an Edge (road) in the network

In [24]:
class Edge:
    def __init__(self, edgeID, nodeA=None, nodeB=None):
        self.__edgeID = edgeID  # ID of the edge
        self.__nodeA = nodeA  # One node connected by this edge
        self.__nodeB = nodeB  # Another node connected by this edge
        if nodeA is not None:
            nodeA.addEdge(self)  # Add this edge to node A
        if nodeB is not None:
            nodeB.addEdge(self)  # Add this edge to node B

    # Get the other node connected to this edge
    def getOtherNode(self, node):
        if self.__nodeA == node:
            return self.__nodeB
        if self.__nodeB == node:
            return self.__nodeA
        return None

    # Get the ID of the edge
    def getID(self):
        return self.__edgeID

    # Get the nodes connected by this edge
    def getNodes(self):
        return (self.__nodeA, self.__nodeB)

    # Add a node to this edge if it's not already connected
    def addNode(self, node):
        if self.__nodeA is None:
            self.__nodeA = node
            node.addEdge(self)
            return True
        elif self.__nodeB is None and self.__nodeA.getID() != node.getID():
            self.__nodeB = node
            node.addEdge(self)
            return True
        return False

    # Check if two edges are equal (based on edge ID)
    def equals(self, edge):
        return edge.getID() == self.getID()

### Class representing a Graph (traffic network) consisting of nodes and edges

In [25]:
class Graph:
    def __init__(self):
        self.__nodes = []  # List of nodes (intersections)
        self.__edges = []  # List of edges (roads)

    def addNode(self, node):
        self.__nodes.append(node)  # Add a node to the graph

    def addNodes(self, nodes):
        self.__nodes.extend(nodes)  # Add multiple nodes to the graph

    def addEdge(self, edge):
        self.__edges.append(edge)  # Add an edge to the graph

    def addEdges(self, edges):
        self.__edges.extend(edges)  # Add multiple edges to the graph

    def getNodes(self):
        return self.__nodes  # Return the list of nodes

    def getEdges(self):
        return self.__edges  # Return the list of edges

    def getNodeWithID(self, nodeID):
        # Find a node in the graph by its ID
        for i in range(len(self.__nodes)):
            if self.__nodes[i].getID() == nodeID:
                return self.__nodes[i]
        return None

    def getEdgeWithId(self, edgeID):
        # Find an edge in the graph by its ID
        for i in range(len(self.__edges)):
            if self.__edges[i].getID() == edgeID:
                return self.__edges[i]
        return None

### Main function to set up the graph, routes, and perform quantum optimization

In [26]:
def main():
    # Open the input file and read lines
    file = open("example.txt", "r")
    lines = file.readlines()
    file.close()

    # Clean up lines by removing comments and empty lines
    for i in range(len(lines) - 1, -1, -1):
        temp = lines[i].split("\n")
        lines[i] = temp[0]
        temp = lines[i].split(" #")
        lines[i] = temp[0]
        temp = lines[i].split("#")
        lines[i] = temp[0]
        if lines[i] == "":
            lines.pop(i)

    # Initialize the graph with nodes
    numOfNodes = int(lines[0])
    graph = Graph()
    for i in range(numOfNodes):
        node = Node(i)
        graph.addNode(node)

    nextLine = 1
    numOfEdges = 0

    # Add edges to the graph based on the input file
    for i in range(numOfNodes):
        nodeIDs = lines[nextLine + i].split(" ")
        for j in range(len(nodeIDs)):
            nodeA = graph.getNodeWithID(i)
            nodeB = graph.getNodeWithID(int(nodeIDs[j]))

            edge_existing = nodeA.getEdgeWithNode(nodeB)
            if edge_existing is not None:
                continue
            edge_existing = nodeB.getEdgeWithNode(nodeA)
            if edge_existing is not None:
                continue

            # Create a new edge if not already existing and add it to the graph
            edge = Edge(numOfEdges, nodeA, nodeB)
            graph.addEdge(edge)
            numOfEdges += 1

    nextLine += numOfNodes
    numOfCars = int(lines[nextLine])  # Get the number of cars
    nextLine += 1

    carRoutes = [[] for _ in range(numOfCars)]  # Initialize car routes for each car

    # Parse car routes from the input file
    for i in range(numOfCars):
        nodeIDs = lines[nextLine + i].split("-")
        route = []
        for j in range(len(nodeIDs) - 1):
            nodeA = graph.getNodeWithID(int(nodeIDs[j]))
            nodeB = graph.getNodeWithID(int(nodeIDs[j + 1]))
            edge = nodeA.getEdgeWithNode(nodeB)
            route.append(edge)
        carRoutes[i].append(route)  # Add the parsed route to the car's route list
    nextLine += numOfCars

    numOfAlternateRoutes = 3  # Define the number of alternate routes for each car

    # Parse alternate routes from the input file
    for i in range(numOfCars * (numOfAlternateRoutes - 1)):
        carNum = int(lines[nextLine + i].split(" ")[0])  # Get the car number
        nodeIDs = (lines[nextLine + i].split(" ")[1]).split("-")  # Get the route nodes
        route = []
        for j in range(len(nodeIDs) - 1):
            nodeA = graph.getNodeWithID(int(nodeIDs[j]))
            nodeB = graph.getNodeWithID(int(nodeIDs[j + 1]))
            edge = nodeA.getEdgeWithNode(nodeB)
            route.append(edge)
        carRoutes[carNum].append(route)  # Add the alternate route to the car's route list

    # Generate the QUBO matrix for the problem
    Q = getQuboMatrix(carRoutes, numOfAlternateRoutes)

    # Use D-Wave's quantum annealer to solve the QUBO problem
    sampler = EmbeddingComposite(DWaveSampler(token='DEV-6f2f442b73ea9653b5c1f0a0af8b4864c35eb9c9'))
    response = sampler.sample_qubo(Q, num_reads=100)

    # Get the results and energy from the quantum solver
    result = list(response.samples())[0]
    resultingEnergy = list(response.data_vectors['energy'])[0]

    # Output the results
    print("roads taken:")
    for i in range(0, len(result)):
        if result[i] == 1:  # If the route is selected, print it
            print(f"car {i//numOfAlternateRoutes + 1}: route {i%numOfAlternateRoutes + 1}")

    print(f"Resulting energy: {resultingEnergy}")  # Output the resulting energy of the solution


# Entry point of the script
if __name__ == "__main__":
    main()


roads taken:
car 1: route 2
car 2: route 1
car 3: route 1
Resulting energy: -31.0
